In [3]:
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save, set_random_seed
import pickle
import os


# api = API(os.environ.get('COMET_ML_API_KEY'))
api = API('IbXp3DT5kVkFPKwnzhg3mgUTL')

set_log_level("ERROR")
set_random_seed(221110)

target_column = 'Total Vertical TBS'

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

for lag in [1,2,3,4,5,6,7,8,9,10,11,12]:

    experiment = Experiment(
    # api_key=os.environ.get('COMET_ML_API_KEY'),
    api_key='IbXp3DT5kVkFPKwnzhg3mgUTL',
    project_name="verticalTBS-tweaking",
    workspace="drdevinhopkins",
)

    params = {
        # growth='off',
        'yearly_seasonality': False,
        'weekly_seasonality': True,
        'daily_seasonality': True,
        'n_lags': lag,
        'n_forecasts': 12,
        'changepoints_range': 0.95,
        'n_changepoints': 50,
        'quantiles': [0.2, 0.5, 0.8]
        # num_hidden_layers=4,
        # d_hidden=36,
        # learning_rate=0.005,
    }

    m = NeuralProphet(**params)
    m = m.add_lagged_regressor(names=regressors)
    m = m.add_country_holidays("CA")
    df_train, df_test = m.split_df(df, valid_p=0.05)

    metrics = m.fit(df_train,
                    freq='H',
                    validation_df=df_test
                    # progress='plot'
                    )
    print(metrics.tail(1))

    experiment.log_parameters(params)
    experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())

    experiment.end()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/verticaltbs-tweaking/72c56e865c6f454b8aa2f575b979218e

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting

  0%|          | 0/159 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([64, 12, 1])) that is different to the input size (torch.Size([64, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([318, 12, 1])) that is different to the input size (torch.Size([318, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)



  0%|          | 0/159 [00:00<?, ?it/s]

COMET INFO: ---------------------------                                                                                                                                              
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/verticaltbs-tweaking/72c56e865c6f454b8aa2f575b979218e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss             : 0.02250959606115462
COMET INFO:     MAE              : 3.919840821137239
COMET INFO:     MAE_val          : 5.537649207983535
COMET INFO:     RMSE             : 5.038371241620564
COMET INFO:     RMSE_val         : 7.242626887921708
COMET INFO:     RegLoss          : 0.0
COMET INFO:     SmoothL1Loss     : 0.016240989541921803
COMET INFO:     SmoothL1Loss_val : 0.03344899333251741
COMET INFO:     loss [3083]      : (0.015120898373425007, 68.13277435302734)
COMET INF

...

COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/verticaltbs-tweaking/98bf778418044aa68651d55bf085ad4f

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralpro

  0%|          | 0/159 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([64, 12, 1])) that is different to the input size (torch.Size([64, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([318, 12, 1])) that is different to the input size (torch.Size([318, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)



  0%|          | 0/159 [00:00<?, ?it/s]

Epoch[81/82]:  99%|█████████▉| 81/82 [06:01<00:04,  4.49s/it, SmoothL1Loss=0.0159, MAE=3.87, RMSE=4.99, Loss=0.0221, RegLoss=0, MAE_val=5.51, RMSE_val=7.21, SmoothL1Loss_val=0.0331]